# Классификация тем статей Ведомостей

В прошлом ноутбуке мы выкачали архив новостей Ведомостей по разным темам. В ссылку на каждую статью зашита её тема. Попробуем по тексту статьи определить её

In [8]:
import numpy as np
import pandas as pd

In [9]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

### Данные

In [10]:
df = pd.read_pickle('data/vedomosti_archive.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'data/vedomosti_archive.pkl'

In [ ]:
df.head()

In [ ]:
df.topic.value_counts()

### Оставим себе задачу бинарной классификации: политика или финансы?

In [ ]:
df = df[(df.topic=='politics')|(df.topic=='finance')]
df.index = range(len(df))

In [ ]:
df["topic"] = df.topic.apply(lambda x: 1 if x=='politics' else 0)

In [ ]:
df.topic.value_counts()

Уберём все токены, которые не являются словами, при этом все числа заменив на "num"

In [ ]:
df["normalized_words"] = df.normalized.apply(lambda x: ' '.join([w if w.isalpha() else 'num' for w in x.split() if w.isalpha() or w.isdigit()]))

### Векторизуем тексты

Здесь мы немного забежим вперёд и используем векторизацию текста в мешок слов ([bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(max_features=5000)

In [ ]:
%%time
X = vectorizer.fit_transform(df.normalized_words).toarray()

In [ ]:
y = df.topic

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
y_train.value_counts()

### Обучим обычным деревом

**Задание:** с помощью кросс-валидации по 10 фолдам и критерием качества f1 score подберите оптимальную глубину дерева в интервале 2-15

*можете переиспользовать код из тетрадки 2, но не забудьте настроить параметры*

In [5]:
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [3]:
parameters_grid = np.arange(2,16)

In [6]:
grid = {'max_depth': parameters_grid}#, 'max_features': features_num}
gridsearch = GridSearchCV(DecisionTreeClassifier(), grid, scoring='f1', cv=10)

In [7]:
%%time
gridsearch.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
height = grdisearch.best_params_ 

### Делаем предсказание

Итак, кросс-валидация помогла нам выбрать глубину дерева. Обучим его с этим знанием

**Задание:** обучите дерево и сделайте предсказание как классов, так и вероятностей принадлежности классам

In [ ]:
tree = DecisionTreeClassifier(max_depth=height)

In [ ]:
tree.predict(X_test)

In [ ]:
tree.predict_proba(X_test)[1]

### Оценим качество

**Задание:** оцените с помощью как минимум 3 метрик качество построенного решения

### Посмотрим на дерево решений

http://www.webgraphviz.com

In [ ]:
from sklearn.tree import export_graphviz

def get_tree_dot_view(clf, feature_names=None, class_names=None):
    print(export_graphviz(clf, out_file=None, filled=True, feature_names=feature_names, class_names=class_names))

In [ ]:
get_tree_dot_view(clf, vectorizer.get_feature_names(), ['finance','politics'])

### Какие слова самые важные?

In [ ]:
def get_top_indexes(s):
    return sorted(range(len(s)), key=lambda k: s[k], reverse=True)

In [ ]:
feature_names = vectorizer.get_feature_names()

In [ ]:
top_indexes = get_top_indexes(clf.feature_importances_)
top_indexes = top_indexes[:10]
top_indexes.reverse()

In [ ]:
top_importances = [clf.feature_importances_[i] for i in top_indexes]
top_words = [feature_names[i] for i in top_indexes]

In [ ]:
plt.barh(np.arange(len(top_importances)), top_importances)
plt.yticks(np.arange(len(top_words)), top_words)
''

посмотрев на дерево решений, видно, что по этим словам просто идёт отсчения из основного корпуса политических текстов в финансовые